In [1]:
from __future__ import division, print_function, unicode_literals
from datetime import datetime
import numpy as np
import os
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Data Prep workflow overview:
Outside Python:
1. In Excel: Compute pareto on occurences of ICD9_PRCDR_CD_1 and ICD9_DGNS_CD_1
2. In Excel: Generate lookup list based on top 80% occurring codes.  This is to avoid having to one hot encode EVERY code (will generate too many columns and end up being even more sparse)
3. Export as csv mappings files ("ICD9_PRCDR_CD_Mappings.csv", "ICD9_DX_CD_Mappings.csv")

In Python
Input files:
- Input claims data file:
  - one row per claim for a given patient for a given hospital admission.  A given patient can have multiple rows correspondong to multiple visits to one or more hospitals
  - contains patient ID, claim dates, upto 10 diagnosis codes, upto 6 procedure codes, hospital (PRVDR_NUM) identifier, physician identifiers (operating, attending, other), various payments amounts, various deductible/liability amounts
- Procedure code mappings file
  - the first 80% occurring ICD9_PRCDR_CD_1 codes are mapped to themselves while the remaining are mapped to "proc_OTHER_PROC"
- Diagnosis code mappings file
  - the first 80% occurring ICD9_DGNS_CD_1 codes are mapped to themselves while the remaining are mapped to "proc_OTHER_DX"
- Beneficiary file to be joined with claims file
  - one row per patient (beneficiary)
  - contains a variety of demographics info and annual summary info on payments etc.  The summary info will be dropped in the analysis notebook as it is not being used in the analysis
  
Data prep steps in python
Pre-Feature Engineering
1. Read in claims data file and clean up
  a. Replace blanks in column names
  b. Convert critical categorical code columns types to string (basestring is what seems to work to support mapping). Code columns are alphanumeric and can be purely numeric so need to be treated as strings
  c. Replace nulls with '0' in procedure and diagnosis codes
2. Read in procedure mappings file and lookup ICD9_PRDCR_CD_1-6 codes in original data with lookups from procedures mapping file to populate columns "Proc1Lookup" - "Proc6Lookup"
3. Generate one hot encoded columns for the unique procedure codes
4. Read in Diagnosis mappings file and replace ICD9_DGNS_CD_1-10 codes in original data with lookups from diagnosis mapping file to populate columns "DX1Lookup" - "DX10Lookup"
5. Generate one-hot-encoded columns for the unique diagnosis codes
6. Replace nulls in numeric payments, liabilities columns ("KPI Columns") with median values
Feature Engineering
7. Compute log of the KPI columns to achieve some level of normalization
8. Populate one-hot-encoded procedure columns with 1 or 0 depending on whether the specific columns occurs ANYWHERE in columns Proc1Lookup through Proc6Lookup
9. Populate one-hot-encoded diagnosis columns with 1 or 0 depending on whether the specific columns occurs ANYWHERE in columns DX1Lookup through DX10Lookup
10. Generate month, week, dayof month, day of week for
    a. Claim start date
    b. Claim end date
11. Read in and merge beneficiary file
12. Compute additional derived variables
   a. age bands in bands of 20 starting as of claim date
13. Print out feature columns file with full set of original, merged and engineered features


outstanding To do
1. Create ohe for PRVDR_NUM

Future To do: Future
1. Consider mapping DX codes to HCC conditions to reduce sparseness
2. Create ohe for DRG_CD

# Pre feature engineering data prep steps

### 1. Read the input file
### 1.a Replace blanks in column names

In [2]:
# 1. Read the input file
data = pd.read_csv('DE1_0_2008_to_2010_Inpatient_Claims_NoHCPCS_Sample_1.csv', dtype=str, nrows=10000)
#1.a Replace blanks in column names
data.columns = data.columns.str.replace(' ', '')
data.dtypes

DESYNPUF_ID                       object
CLM_ID                            object
SEGMENT                           object
CLM_FROM_DT                       object
CLM_THRU_DT                       object
PRVDR_NUM                         object
CLM_PMT_AMT                       object
NCH_PRMRY_PYR_CLM_PD_AMT          object
AT_PHYSN_NPI                      object
OP_PHYSN_NPI                      object
OT_PHYSN_NPI                      object
CLM_ADMSN_DT                      object
ADMTNG_ICD9_DGNS_CD               object
CLM_PASS_THRU_PER_DIEM_AMT        object
NCH_BENE_IP_DDCTBL_AMT            object
NCH_BENE_PTA_COINSRNC_LBLTY_AM    object
NCH_BENE_BLOOD_DDCTBL_LBLTY_AM    object
CLM_UTLZTN_DAY_CNT                object
NCH_BENE_DSCHRG_DT                object
CLM_DRG_CD                        object
ICD9_DGNS_CD_1                    object
ICD9_DGNS_CD_2                    object
ICD9_DGNS_CD_3                    object
ICD9_DGNS_CD_4                    object
ICD9_DGNS_CD_5  

In [3]:
#1.b convert code columns to string type
data['ICD9_DGNS_CD_1'] = data['ICD9_DGNS_CD_1'].astype(basestring)
data['ICD9_DGNS_CD_2'] = data['ICD9_DGNS_CD_2'].astype(basestring)
data['ICD9_DGNS_CD_3'] = data['ICD9_DGNS_CD_3'].astype(basestring)
data['ICD9_DGNS_CD_4'] = data['ICD9_DGNS_CD_4'].astype(basestring)
data['ICD9_DGNS_CD_5'] = data['ICD9_DGNS_CD_5'].astype(basestring)
data['ICD9_DGNS_CD_6'] = data['ICD9_DGNS_CD_6'].astype(basestring)
data['ICD9_DGNS_CD_7'] = data['ICD9_DGNS_CD_7'].astype(basestring)
data['ICD9_DGNS_CD_8'] = data['ICD9_DGNS_CD_8'].astype(basestring)
data['ICD9_DGNS_CD_9'] = data['ICD9_DGNS_CD_9'].astype(basestring)
data['ICD9_DGNS_CD_10'] = data['ICD9_DGNS_CD_10'].astype(basestring)

data['ICD9_PRCDR_CD_1'] = data['ICD9_PRCDR_CD_1'].astype(basestring)
data['ICD9_PRCDR_CD_2'] = data['ICD9_PRCDR_CD_2'].astype(basestring)
data['ICD9_PRCDR_CD_3'] = data['ICD9_PRCDR_CD_3'].astype(basestring)
data['ICD9_PRCDR_CD_4'] = data['ICD9_PRCDR_CD_4'].astype(basestring)
data['ICD9_PRCDR_CD_5'] = data['ICD9_PRCDR_CD_5'].astype(basestring)
data['ICD9_PRCDR_CD_6'] = data['ICD9_PRCDR_CD_6'].astype(basestring)
#data.dtypes

NameError: name 'basestring' is not defined

In [4]:
data.describe(include='all')

data['ICD9_PRCDR_CD_1'].head(10)
data['ICD9_PRCDR_CD_2'].head(10)
data['ICD9_PRCDR_CD_3'].head(10)
data['ICD9_PRCDR_CD_4'].head(10)
data['ICD9_PRCDR_CD_5'].head(10)
data['ICD9_PRCDR_CD_6'].head(10)

data['ICD9_DGNS_CD_1'].head(10)
data['ICD9_DGNS_CD_2'].head(10)
data['ICD9_DGNS_CD_3'].head(10)
data['ICD9_DGNS_CD_4'].head(10)
data['ICD9_DGNS_CD_5'].head(10)
data['ICD9_DGNS_CD_6'].head(10)
data['ICD9_DGNS_CD_7'].head(10)
data['ICD9_DGNS_CD_8'].head(10)
data['ICD9_DGNS_CD_9'].head(10)
data['ICD9_DGNS_CD_10'].head(10)

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,...,ICD9_DGNS_CD_7,ICD9_DGNS_CD_8,ICD9_DGNS_CD_9,ICD9_DGNS_CD_10,ICD9_PRCDR_CD_1,ICD9_PRCDR_CD_2,ICD9_PRCDR_CD_3,ICD9_PRCDR_CD_4,ICD9_PRCDR_CD_5,ICD9_PRCDR_CD_6
count,10000,10000,10000,9988,9988,10000,10000,10000,9912,5831,...,8078,7393,6628,773,5690,3306,2155,1375,965,690
unique,5626,9988,2,1099,1088,1842,75,55,4771,3225,...,1295,1268,1203,378,658,801,673,495,395,306
top,1426B87573531324,196861177018065,1,20080724,20080602,23006G,4000,0,9011551271,9612910514,...,4019,4019,4019,4019,9904,4019,4019,4019,4019,4019
freq,11,2,9988,23,25,125,1150,9738,79,50,...,374,287,266,24,269,233,114,80,37,33


0     NaN
1     NaN
2    7092
3     NaN
4     NaN
5     NaN
6    8659
7     NaN
8     NaN
9      73
Name: ICD9_PRCDR_CD_1, dtype: object

0      NaN
1      NaN
2     6186
3      NaN
4    E8889
5      NaN
6      NaN
7      NaN
8      NaN
9      311
Name: ICD9_PRCDR_CD_2, dtype: object

0      NaN
1      NaN
2    V5866
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
Name: ICD9_PRCDR_CD_3, dtype: object

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
Name: ICD9_PRCDR_CD_4, dtype: object

...

0     4019
1    71590
2      NaN
3    32723
4     2639
5    V4581
6    42731
7     4659
8     5845
9     2720
Name: ICD9_DGNS_CD_6, dtype: object

0    V4502
1     2724
2      NaN
3      NaN
4     7840
5     4019
6    78729
7      NaN
8     4019
9    53081
Name: ICD9_DGNS_CD_7, dtype: object

0    73300
1    19889
2      NaN
3      NaN
4     7856
5      NaN
6     V103
7      NaN
8     4189
9    V5866
Name: ICD9_DGNS_CD_8, dtype: object

0    E9330
1     5849
2      NaN
3      NaN
4     4271
5      NaN
6    34290
7      NaN
8    78729
9     4254
Name: ICD9_DGNS_CD_9, dtype: object

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    496
Name: ICD9_DGNS_CD_10, dtype: object

1. read in lookup table
2. get unique values
3. for each entry in unique list, create empty column
4. for each row in Df, step through prcdr columns and update ohe column to 1 if present (do no increment)

In [5]:
data.shape

(10000, 36)

### 1.c Replace nulls in procedure and diagnosis codes with '0'

In [6]:
data.dtypes

data['ICD9_PRCDR_CD_1'] = data['ICD9_PRCDR_CD_1'].fillna('0')
data["ICD9_PRCDR_CD_2"] = data["ICD9_PRCDR_CD_2"].fillna('0')
data["ICD9_PRCDR_CD_3"] = data["ICD9_PRCDR_CD_3"].fillna('0')
data["ICD9_PRCDR_CD_4"] = data["ICD9_PRCDR_CD_4"].fillna('0')
data["ICD9_PRCDR_CD_5"] = data["ICD9_PRCDR_CD_5"].fillna('0')
data["ICD9_PRCDR_CD_6"] = data["ICD9_PRCDR_CD_6"].fillna('0')

data['ICD9_DGNS_CD_1'] = data['ICD9_DGNS_CD_1'].fillna('0')
data['ICD9_DGNS_CD_2'] = data['ICD9_DGNS_CD_2'].fillna('0')
data['ICD9_DGNS_CD_3'] = data['ICD9_DGNS_CD_3'].fillna('0')
data['ICD9_DGNS_CD_4'] = data['ICD9_DGNS_CD_4'].fillna('0')
data['ICD9_DGNS_CD_5'] = data['ICD9_DGNS_CD_5'].fillna('0')
data['ICD9_DGNS_CD_6'] = data['ICD9_DGNS_CD_6'].fillna('0')
data['ICD9_DGNS_CD_7'] = data['ICD9_DGNS_CD_7'].fillna('0')
data['ICD9_DGNS_CD_8'] = data['ICD9_DGNS_CD_8'].fillna('0')
data['ICD9_DGNS_CD_9'] = data['ICD9_DGNS_CD_9'].fillna('0')
data['ICD9_DGNS_CD_10'] = data['ICD9_DGNS_CD_10'].fillna('0')

data.shape


DESYNPUF_ID                       object
CLM_ID                            object
SEGMENT                           object
CLM_FROM_DT                       object
CLM_THRU_DT                       object
PRVDR_NUM                         object
CLM_PMT_AMT                       object
NCH_PRMRY_PYR_CLM_PD_AMT          object
AT_PHYSN_NPI                      object
OP_PHYSN_NPI                      object
OT_PHYSN_NPI                      object
CLM_ADMSN_DT                      object
ADMTNG_ICD9_DGNS_CD               object
CLM_PASS_THRU_PER_DIEM_AMT        object
NCH_BENE_IP_DDCTBL_AMT            object
NCH_BENE_PTA_COINSRNC_LBLTY_AM    object
NCH_BENE_BLOOD_DDCTBL_LBLTY_AM    object
CLM_UTLZTN_DAY_CNT                object
NCH_BENE_DSCHRG_DT                object
CLM_DRG_CD                        object
ICD9_DGNS_CD_1                    object
ICD9_DGNS_CD_2                    object
ICD9_DGNS_CD_3                    object
ICD9_DGNS_CD_4                    object
ICD9_DGNS_CD_5  

(10000, 36)

### 2. Read in procedure mappings file and lookup ICD9_PRDCR_CD_1-6 codes in original data with lookups from procedures mapping file to populate columns "Proc1Lookup" - "Proc6Lookup"
### 3. Generate one hot encoded columns for the unique procedure codes
### 4. Read in Diagnosis mappings file and replace ICD9_DGNS_CD_1-10 codes in original data with lookups from diagnosis mapping file to populate columns "DX1Lookup" - "DX10Lookup"
### 5. Generate one-hot-encoded columns for the unique diagnosis codes

In [7]:
#2. Read in procedure mappings file and lookup ICD9_PRDCR_CD_1-6 codes in original data with lookups from procedures mapping file to populate columns "Proc1Lookup" - "Proc6Lookup"
proc_map = pd.read_csv('ICD9_PRCDR_CD_Mappings.csv',skipinitialspace=True).astype(str)
proc_unique = proc_map.Lookup.unique().astype(str)

#3. Generate one hot encoded columns for the unique procedure codes 
for index, row in data.iterrows():
  for j in range(len(proc_unique)):
    data['proc_'+str(proc_unique[j])]=0


cmap = proc_map.set_index('ICD9_PRCDR_CD_1')['Lookup']

data['Proc1Lookup'] = data['ICD9_PRCDR_CD_1'].map(cmap, na_action='ignore')
data['Proc2Lookup'] = data['ICD9_PRCDR_CD_2'].map(cmap, na_action='ignore')
data['Proc3Lookup'] = data['ICD9_PRCDR_CD_3'].map(cmap, na_action='ignore')
data['Proc4Lookup'] = data['ICD9_PRCDR_CD_4'].map(cmap, na_action='ignore')
data['Proc5Lookup'] = data['ICD9_PRCDR_CD_5'].map(cmap, na_action='ignore')
data['Proc6Lookup'] = data['ICD9_PRCDR_CD_6'].map(cmap, na_action='ignore')

      
#4. Read in Diagnosis mappings file and replace ICD9_DGNS_CD_1-10 codes in original data with lookups from diagnosis mapping file to populate columns "DX1Lookup" - "DX10Lookup"
dx_map = pd.read_csv('ICD9_DX_CD_Mappings.csv',skipinitialspace=True).astype(str)
dx_unique = dx_map.Lookup.unique().astype(str)

#5. Generate one-hot-encoded columns for the unique diagnosis codes
for index, row in data.iterrows():
  for j in range(len(dx_unique)):
    data['dx_'+str(dx_unique[j])]=0
    
dmap = dx_map.set_index('ICD9_DGNS_CD_1')['Lookup']

data['Dx1Lookup'] = data['ICD9_DGNS_CD_1'].map(dmap, na_action='ignore')
data['Dx2Lookup'] = data['ICD9_DGNS_CD_2'].map(dmap, na_action='ignore')
data['Dx3Lookup'] = data['ICD9_DGNS_CD_3'].map(dmap, na_action='ignore')
data['Dx4Lookup'] = data['ICD9_DGNS_CD_4'].map(dmap, na_action='ignore')
data['Dx5Lookup'] = data['ICD9_DGNS_CD_5'].map(dmap, na_action='ignore')
data['Dx6Lookup'] = data['ICD9_DGNS_CD_6'].map(dmap, na_action='ignore')
data['Dx7Lookup'] = data['ICD9_DGNS_CD_7'].map(dmap, na_action='ignore')
data['Dx8Lookup'] = data['ICD9_DGNS_CD_8'].map(dmap, na_action='ignore')
data['Dx9Lookup'] = data['ICD9_DGNS_CD_9'].map(dmap, na_action='ignore')
data['Dx10Lookup'] = data['ICD9_DGNS_CD_10'].map(dmap, na_action='ignore')
        
data.dtypes
data.head(10)
proc_map.dtypes
proc_map.head(5)

'''
#debugging prints (uncomment as needed)
data['Dx1Lookup'].head(10)
data['Dx2Lookup'].head(10)
data['Dx3Lookup'].head(10)
data['ICD9_DGNS_CD_1'].head(10)
data['ICD9_DGNS_CD_2'].head(10)
data['ICD9_DGNS_CD_3'].head(10)
first5pairs = {k: dmap[k] for k in dmap.keys()[:5]}
first5pairs

data.shape

#debugging prints
proc_map.dtypes
proc_map.head(5)

data['Proc1Lookup'].head(10)
data['Proc2Lookup'].head(10)
data['Proc3Lookup'].head(10)
data['ICD9_PRCDR_CD_1'].head(10)
data['ICD9_PRCDR_CD_2'].head(10)
data['ICD9_PRCDR_CD_3'].head(10)

first5pairs = {k: cmap[k] for k in cmap.keys()[:5]}
first5pairs
'''

DESYNPUF_ID                       object
CLM_ID                            object
SEGMENT                           object
CLM_FROM_DT                       object
CLM_THRU_DT                       object
PRVDR_NUM                         object
CLM_PMT_AMT                       object
NCH_PRMRY_PYR_CLM_PD_AMT          object
AT_PHYSN_NPI                      object
OP_PHYSN_NPI                      object
OT_PHYSN_NPI                      object
CLM_ADMSN_DT                      object
ADMTNG_ICD9_DGNS_CD               object
CLM_PASS_THRU_PER_DIEM_AMT        object
NCH_BENE_IP_DDCTBL_AMT            object
NCH_BENE_PTA_COINSRNC_LBLTY_AM    object
NCH_BENE_BLOOD_DDCTBL_LBLTY_AM    object
CLM_UTLZTN_DAY_CNT                object
NCH_BENE_DSCHRG_DT                object
CLM_DRG_CD                        object
ICD9_DGNS_CD_1                    object
ICD9_DGNS_CD_2                    object
ICD9_DGNS_CD_3                    object
ICD9_DGNS_CD_4                    object
ICD9_DGNS_CD_5  

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,...,Dx1Lookup,Dx2Lookup,Dx3Lookup,Dx4Lookup,Dx5Lookup,Dx6Lookup,Dx7Lookup,Dx8Lookup,Dx9Lookup,Dx10Lookup
0,00013D2EFD8E45D1,196661176988405,1,20100312,20100313,2600GD,4000,0,3139083564,NaN,...,7802,OTHER_DX,NaN,4280,NaN,4019,NaN,NaN,NaN,NaN
1,00016F745862898F,196201177000368,1,20090412,20090418,3900MB,26000,0,6476809087,NaN,...,1970,4019,OTHER_DX,OTHER_DX,2768,OTHER_DX,NaN,OTHER_DX,5849,NaN
2,00016F745862898F,196661177015632,1,20090831,20090902,3900HM,5000,0,611998537,611998537,...,OTHER_DX,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00016F745862898F,196091176981058,1,20090917,20090920,3913XU,5000,0,4971602784,NaN,...,OTHER_DX,OTHER_DX,NaN,34590,NaN,OTHER_DX,NaN,NaN,NaN,NaN
4,00016F745862898F,196261176983265,1,20100626,20100701,3900MB,16000,0,6408400473,1960859579,...,OTHER_DX,4019,OTHER_DX,NaN,OTHER_DX,OTHER_DX,7840,OTHER_DX,4271,NaN
5,00052705243EA128,196991176971757,1,20080912,20080912,1401HG,14000,0,6132010904,NaN,...,486,OTHER_DX,42731,42830,NaN,NaN,4019,NaN,NaN,NaN
6,0007F12A492FD25D,196661176963773,1,20080919,20080922,3400WD,5000,0,8956735757,6551008003,...,OTHER_DX,53550,42820,496,NaN,42731,OTHER_DX,NaN,OTHER_DX,NaN
7,0007F12A492FD25D,196821177025734,1,20100602,20100606,34S0KQ,5000,0,5838958809,NaN,...,49121,25002,OTHER_DX,4111,4280,OTHER_DX,NaN,NaN,NaN,NaN
8,0007F12A492FD25D,196551177025145,1,20100522,20100612,3400WD,14000,0,1907446990,NaN,...,V5789,OTHER_DX,OTHER_DX,2761,OTHER_DX,5845,4019,NaN,OTHER_DX,NaN
9,0007F12A492FD25D,196831176966961,1,20100616,20100619,3401ZP,29000,0,4959466403,8002609409,...,OTHER_DX,NaN,OTHER_DX,NaN,NaN,NaN,53081,NaN,4254,496


ICD9_PRCDR_CD_1    object
Lookup             object
dtype: object

,ICD9_PRCDR_CD_1,Lookup
0,9904,9904
1,8154,8154
2,3893,3893
3,66,66
4,3995,3995


"\n#debugging prints (uncomment as needed)\ndata['Dx1Lookup'].head(10)\ndata['Dx2Lookup'].head(10)\ndata['Dx3Lookup'].head(10)\ndata['ICD9_DGNS_CD_1'].head(10)\ndata['ICD9_DGNS_CD_2'].head(10)\ndata['ICD9_DGNS_CD_3'].head(10)\nfirst5pairs = {k: dmap[k] for k in dmap.keys()[:5]}\nfirst5pairs\n\ndata.shape\n\n#debugging prints\nproc_map.dtypes\nproc_map.head(5)\n\ndata['Proc1Lookup'].head(10)\ndata['Proc2Lookup'].head(10)\ndata['Proc3Lookup'].head(10)\ndata['ICD9_PRCDR_CD_1'].head(10)\ndata['ICD9_PRCDR_CD_2'].head(10)\ndata['ICD9_PRCDR_CD_3'].head(10)\n\nfirst5pairs = {k: cmap[k] for k in cmap.keys()[:5]}\nfirst5pairs\n"

### 6. Replace nulls in numeric payments, liabilities columns ("KPI Columns") with median values

In [8]:
#Find null values in KPI columns
#print(data["ICD9_PRCDR_CD_1"].isnull().sum())
print(data["CLM_PMT_AMT"].isnull().sum())
print(data["NCH_PRMRY_PYR_CLM_PD_AMT"].isnull().sum())
print(data["CLM_PASS_THRU_PER_DIEM_AMT"].isnull().sum())
print(data["NCH_BENE_IP_DDCTBL_AMT"].isnull().sum())
print(data["NCH_BENE_PTA_COINSRNC_LBLTY_AM"].isnull().sum())
print(data["NCH_BENE_BLOOD_DDCTBL_LBLTY_AM"].isnull().sum())
print(data["CLM_UTLZTN_DAY_CNT"].isnull().sum())
print(data["CLM_FROM_DT"].isnull().sum())
data.shape

0
0
0
322
0
0
12
12


(10000, 523)

In [9]:
data.dropna(subset = ['CLM_FROM_DT'], inplace=True)
#Set null values to median
median = data["CLM_PMT_AMT"].median()
data["CLM_PMT_AMT"].fillna(median, inplace=True)
median = data["NCH_PRMRY_PYR_CLM_PD_AMT"].median()
data["NCH_PRMRY_PYR_CLM_PD_AMT"].fillna(median, inplace=True)
median = data["CLM_PASS_THRU_PER_DIEM_AMT"].median()
data["CLM_PASS_THRU_PER_DIEM_AMT"].fillna(median, inplace=True)
median = data["NCH_BENE_IP_DDCTBL_AMT"].median()
data["NCH_BENE_IP_DDCTBL_AMT"].fillna(median, inplace=True)
median = data["NCH_BENE_PTA_COINSRNC_LBLTY_AM"].median()
data["NCH_BENE_PTA_COINSRNC_LBLTY_AM"].fillna(median, inplace=True)
median = data["NCH_BENE_BLOOD_DDCTBL_LBLTY_AM"].median()
data["NCH_BENE_BLOOD_DDCTBL_LBLTY_AM"].fillna(median, inplace=True)
median = data["CLM_UTLZTN_DAY_CNT"].median()
data["CLM_UTLZTN_DAY_CNT"].fillna(median, inplace=True)

# Feature Engineering

### 7. Compute log of the KPI columns to achieve some level of normalization

In [10]:
data['CLM_PMT_AMT'] = data['CLM_PMT_AMT'].astype(float)
data['NCH_PRMRY_PYR_CLM_PD_AMT'] = data['NCH_PRMRY_PYR_CLM_PD_AMT'].astype(float)
data['CLM_PASS_THRU_PER_DIEM_AMT'] = data['CLM_PASS_THRU_PER_DIEM_AMT'].astype(float)
data['NCH_BENE_IP_DDCTBL_AMT'] = data['NCH_BENE_IP_DDCTBL_AMT'].astype(float)
data['NCH_BENE_PTA_COINSRNC_LBLTY_AM'] = data['NCH_BENE_PTA_COINSRNC_LBLTY_AM'].astype(float)
data['NCH_BENE_BLOOD_DDCTBL_LBLTY_AM'] = data['NCH_BENE_BLOOD_DDCTBL_LBLTY_AM'].astype(float)
#compute Log of KPIs
data["LOG_CLM_PMT_AMT"] = np.log(1+data.CLM_PMT_AMT)
data["LOG_NCH_PRMRY_PYR_CLM_PD_AMT"]  = np.log(1+data.NCH_PRMRY_PYR_CLM_PD_AMT)
data["LOG_CLM_PASS_THRU_PER_DIEM_AMT"]  = np.log(1+data.CLM_PASS_THRU_PER_DIEM_AMT)
data["LOG_NCH_BENE_IP_DDCTBL_AMT"]  = np.log(1+data.NCH_BENE_IP_DDCTBL_AMT)
data["LOG_NCH_BENE_PTA_COINSRNC_LBLTY_AM"] = np.log(1+data.NCH_BENE_PTA_COINSRNC_LBLTY_AM)
data["LOG_NCH_BENE_BLOOD_DDCTBL_LBLTY_AM"] = np.log(1+data.NCH_BENE_BLOOD_DDCTBL_LBLTY_AM)

data.shape

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


(9988, 529)

### 8. Populate one-hot-encoded procedure columns with 1 or 0 depending on whether the specific columns occurs ANYWHERE in columns Proc1Lookup through Proc6Lookup

In [11]:
data[['proc_311','proc_8659','proc_OTHER_PROC']].dtypes
for index, row in data.iterrows():
    code = str(row['Proc1Lookup'])
    if code <> 'nan':
        data.loc[index, 'proc_'+ code] = 1 
#    else:
#        data.loc[index, 'proc_OTHER_P'+ code] = 0.0
#    data['proc_'+ str(code)].fillna(value=0.0,inplace=True)


    code = str(row['Proc2Lookup'])
    if code <> 'nan':
        data.loc[index, 'proc_'+ str(code)] = 1    
#    else:
#        data.loc[index, 'proc_'+ str(code)] = 0.0
#    data['proc_'+ str(code)].fillna(value=0.0,inplace=True)

    code = str(row['Proc3Lookup'])
    if code <> 'nan':
        data.loc[index, 'proc_'+ str(code)] = 1    
        
    code = str(row['Proc4Lookup'])
    if code <> 'nan':
        data.loc[index, 'proc_'+ str(code)] = 1    
        
    code = str(row['Proc5Lookup'])
    if code <> 'nan':
        data.loc[index, 'proc_'+ str(code)] = 1    
        
    code = str(row['Proc6Lookup'])
    if code <> 'nan':
        data.loc[index, 'proc_'+ str(code)] = 1  




SyntaxError: invalid syntax (<ipython-input-11-cccf87797458>, line 4)

In [ ]:
'''
#Check Proc assignments
data.shape
data[['proc_311','proc_8659','proc_OTHER_PROC']].dtypes
data['proc_8659'].head(10)
data['proc_311'].head(10)
data['proc_OTHER_PROC'].head(10)
'''

### 9. Populate one-hot-encoded diagnosis columns with 1 or 0 depending on whether the specific columns occurs ANYWHERE in columns DX1Lookup through DX6Lookup

In [ ]:
for index, row in data.iterrows():
    code = str(row['Dx1Lookup'])
    if code <> 'nan':
        data.loc[index, 'dx_'+ str(code)] = 1 
#    else:
#        data.loc[index, 'dx_'+ str(code)] = 0.0
#    data['dx_'+ str(code)].fillna(value=0.0,inplace=True)

    code = str(row['Dx2Lookup'])
    if code <> 'nan':
        data.loc[index, 'dx_'+ str(code)] = 1    

    code = str(row['Dx3Lookup'])
    if code <> 'nan':
        data.loc[index, 'dx_'+ str(code)] = 1    
        
    code = str(row['Dx4Lookup'])
    if code <> 'nan':
        data.loc[index, 'dx_'+ str(code)] = 1    
        
    code = str(row['Dx5Lookup'])
    if code <> 'nan':
        data.loc[index, 'dx_'+ str(code)] = 1    
        
    code = str(row['Dx6Lookup'])
    if code <> 'nan':
        data.loc[index, 'dx_'+ str(code)] = 1  

    code = str(row['Dx7Lookup'])
    if code <> 'nan':
        data.loc[index, 'dx_'+ str(code)] = 1  

    code = str(row['Dx8Lookup'])
    if code <> 'nan':
        data.loc[index, 'dx_'+ str(code)] = 1  

    code = str(row['Dx9Lookup'])
    if code <> 'nan':
        data.loc[index, 'dx_'+ str(code)] = 1  

    code = str(row['Dx10Lookup'])
    if code <> 'nan':
        data.loc[index, 'dx_'+ str(code)] = 1      

In [ ]:
#Check Dx assignments
data.shape
data[['dx_7802','dx_4019','dx_OTHER_DX']].dtypes
data['dx_7802'].head(10)
data['dx_4019'].head(10)
data['dx_OTHER_DX'].head(10)

In [ ]:
#drop the lookup columns as they have served their purpose
data = data.drop(['Proc1Lookup','Proc2Lookup','Proc3Lookup','Proc4Lookup','Proc5Lookup','Proc6Lookup'], axis=1)
data = data.drop(['Dx1Lookup','Dx2Lookup','Dx3Lookup','Dx4Lookup','Dx5Lookup','Dx6Lookup','Dx7Lookup','Dx8Lookup','Dx9Lookup','Dx10Lookup'], axis=1)
data.shape

### 10. Generate month, week, dayof month, day of week for
    a. Claim start date
    b. Claim end date

In [ ]:
#Create year, month, week, day of week columns for dates
data['CLM_FROM_DT'].head(5)
data['CLM_FROM_DT'] = pd.to_datetime(data['CLM_FROM_DT'].astype(str),format='%Y%m%d')
data['CLM_THRU_DT'] = pd.to_datetime(data['CLM_THRU_DT'].astype(str),format='%Y%m%d')
data['CLM_ADMSN_DT'] = pd.to_datetime(data['CLM_ADMSN_DT'].astype(str),format='%Y%m%d')
data['CLM_FROM_DT'].head(5)



In [ ]:
print(data['CLM_FROM_DT'].isnull().sum())
data['CLM_FROM_Year'] = data['CLM_FROM_DT'].dt.year
data['CLM_FROM_Month'] = data['CLM_FROM_DT'].dt.month
data['CLM_FROM_Day'] = data['CLM_FROM_DT'].dt.day
data['CLM_FROM_DOW'] = data['CLM_FROM_DT'].dt.dayofweek

data.dropna(subset = ['CLM_FROM_DT','CLM_FROM_Year','CLM_FROM_Month','CLM_FROM_Day','CLM_FROM_DOW'])

data['CLM_FROM_DT'].head(5)
data['CLM_FROM_Year'].head(5)
data['CLM_FROM_Month'].head(5)
data['CLM_FROM_Day'].head(5)
data['CLM_FROM_DOW'].head(5)

print(data['CLM_FROM_Year'].isna().sum())
print(data['CLM_FROM_Month'].isnull().sum())
print(data['CLM_FROM_Day'].isnull().sum())
print(data['CLM_FROM_DOW'].isnull().sum())
print(data['CLM_FROM_DT'].isnull().sum())

### 11. Read in and merge beneficiary file

In [ ]:
#Read in demo info
demo_info = pd.read_csv('DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv')
data.shape
demo_info.shape
#Compute derived demo variables
demo_info['BENE_BIRTH_DT'] = pd.to_datetime(demo_info['BENE_BIRTH_DT'].astype(str),format='%Y%m%d')
demo_info.head(10)




In [ ]:
data.head(10)
demo_info.head(10)
merged_data = pd.merge(data, demo_info, on='DESYNPUF_ID')
merged_data.shape
merged_data.head(10)


###  12. Compute additional derived variables
   a. age bands in bands of 20 starting as of claim date

In [ ]:
#compute derived features
#1. Age group
merged_data['BENE_Age_Group'] = ((merged_data['CLM_FROM_DT'] - merged_data['BENE_BIRTH_DT']).dt.days / (20*365)).round(0)
merged_data['BENE_BIRTH_DT'].head(10)
merged_data['CLM_FROM_DT'].head(10)
merged_data['BENE_Age_Group'].head(10)

### 13. Print out feature columns file with full set of original, merged and engineered features

In [ ]:
merged_data.dtypes
merged_data.to_csv('Demo_CMS_IP_features_columns.csv',index=False, header=True)